In [1]:
import logging

import probtorch
import torch

import combinators
import hmm
import importance
import mcmc
import utils

logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

In [2]:
generative = combinators.BroadcastingTrace(1)

In [3]:
hmm_params = utils.vardict({
    'mu': {
        'loc': torch.arange(5, dtype=torch.float) * 2,
        'scale': torch.ones(5) * 0.25,
    },
    'sigma': {
        'loc': torch.ones(5),
        'scale': torch.ones(5) * 0.25,
    }
})
for k in range(6):
    hmm_params['Pi_%d' % k] = {'concentration': torch.ones(5)}

In [4]:
init_hmm = combinators.Model(f=hmm.init_hmm, hyper=hmm_params)

In [5]:
z0, mu, sigma, pi, pi0 = init_hmm(trace=generative)

In [6]:
hmm_step = combinators.Model(f=hmm.hmm_step)
hmm_run = combinators.Model.sequence(hmm_step, 50, z0, mu, sigma, pi, pi0)

In [7]:
z_last, mu, sigma, pi, pi0 = hmm_run(trace=generative)

In [8]:
data = generative.unwrap(lambda k, rv: 'X_' in k)

In [9]:
num_particles = 250

In [10]:
smc_hmm_params = utils.vardict({
    'mu': {
        'loc': torch.arange(5, dtype=torch.float) * 2,
        'scale': torch.ones(5),
    },
    'sigma': {
        'loc': torch.ones(5),
        'scale': torch.ones(5),
    }
})
for k in range(6):
    smc_hmm_params['Pi_%d' % k] = {'concentration': torch.ones(5)}

In [11]:
init_hmm = combinators.Model(hmm.init_hmm, trainable=smc_hmm_params)

In [12]:
smc_hmm = importance.ImportanceResampler.smc(combinators.Model(hmm.hmm_step), 50, initializer=init_hmm)

In [13]:
hmm_proposal = combinators.Model.compose(combinators.Model.sequence(combinators.Model(hmm.hmm_step), 50), init_hmm, intermediate_name='initializer')

In [14]:
particle_mh = mcmc.IndependentMH(smc_hmm, hmm_proposal, num_particles*10)

In [15]:
samples, elbos = particle_mh(trace=importance.ResamplerTrace(num_particles, data=data))
inference = particle_mh.trace

In [16]:
for t in range(50):
    t += 1
    key = 'Z_%d' % t
    accuracy = (inference[key].value == generative[key].value).sum().to(dtype=torch.float) / inference.num_particles
    print('SMC accuracy at time %d: %f' % (t, accuracy))

SMC accuracy at time 1: 0.000000
SMC accuracy at time 2: 0.124000
SMC accuracy at time 3: 0.000000
SMC accuracy at time 4: 0.200000
SMC accuracy at time 5: 0.324000
SMC accuracy at time 6: 0.200000
SMC accuracy at time 7: 1.000000
SMC accuracy at time 8: 1.000000
SMC accuracy at time 9: 1.000000
SMC accuracy at time 10: 0.480000
SMC accuracy at time 11: 0.356000
SMC accuracy at time 12: 0.320000
SMC accuracy at time 13: 0.000000
SMC accuracy at time 14: 0.380000
SMC accuracy at time 15: 0.000000
SMC accuracy at time 16: 0.124000
SMC accuracy at time 17: 0.696000
SMC accuracy at time 18: 0.324000
SMC accuracy at time 19: 0.064000
SMC accuracy at time 20: 0.064000
SMC accuracy at time 21: 0.360000
SMC accuracy at time 22: 0.240000
SMC accuracy at time 23: 0.644000
SMC accuracy at time 24: 0.256000
SMC accuracy at time 25: 0.060000
SMC accuracy at time 26: 0.124000
SMC accuracy at time 27: 0.356000
SMC accuracy at time 28: 0.488000
SMC accuracy at time 29: 0.000000
SMC accuracy at time 30

In [17]:
elbos

tensor([-136.5953, -136.5953, -136.5953,  ..., -116.5143, -116.5143,
        -116.5143])